In [1]:
DATA_NAME = 'standardTesting-full' 
TRANSFORM = 'wavelet-vertical'
CHANNEL = 'gray'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_gray_vertical_wavelet_full_standardTesting.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        if len(group_data_map[group]) < 100:
            X[:, i] = np.full(bootstrap_size, np.nan)
        else:
            X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df = cov_df.dropna(how='all').dropna(axis=1, how='all')
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,4,5,6,7,8,9,10
4,1117.16,0.01,0.01,0.0,-0.00,-0.00,0.00
5,0.01,296.33,0.00,0.0,-0.00,-0.00,-0.00
6,0.01,0.00,58.39,0.0,-0.00,0.00,-0.00
7,0.00,0.00,0.00,10.9,0.00,-0.00,-0.00
8,-0.00,-0.00,-0.00,0.0,2.09,-0.00,-0.00
9,-0.00,-0.00,0.00,-0.0,-0.00,0.38,0.00
10,0.00,-0.00,-0.00,-0.0,-0.00,0.00,0.06


In [10]:
corr_matrix = cov_df / np.sqrt(np.outer(np.diag(cov_df), np.diag(cov_df)))
corr_matrix.round(5)

,4,5,6,7,8,9,10
4,1.00000,0.00002,0.00003,0.00002,-0.00000,-0.00003,0.00001
5,0.00002,1.00000,0.00002,0.00003,-0.00001,-0.00005,-0.00004
6,0.00003,0.00002,1.00000,0.00003,-0.00001,0.00002,-0.00002
7,0.00002,0.00003,0.00003,1.00000,0.00002,-0.00001,-0.00005
8,-0.00000,-0.00001,-0.00001,0.00002,1.00000,-0.00003,-0.00003
9,-0.00003,-0.00005,0.00002,-0.00001,-0.00003,1.00000,0.00001
10,0.00001,-0.00004,-0.00002,-0.00005,-0.00003,0.00001,1.00000


In [11]:
np.linalg.norm(cov_df - np.diag(np.diag(cov_df)))  # Remove diagonal elements for visualization

0.020448205262667593

In [12]:
pca = PCA()
pca.fit(cov_df)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[1.78679112e+05 1.21913066e+04 4.54859972e+02 1.48776446e+01
 4.89084433e-01 1.19513421e-02 3.54798173e-34]

Principal components (eigenvectors):


,0,1,2,3,4,5,6
0,0.998840,-0.047354,-0.008589,-0.001598,-0.000307,-0.000056,-0.000009
1,0.046948,0.998014,-0.041335,-0.007364,-0.001415,-0.000255,-0.000040
2,0.010419,0.040470,0.997907,-0.048509,-0.008933,-0.001598,-0.000252
3,0.002413,0.009107,0.047434,0.996545,-0.066536,-0.011371,-0.001791
4,0.000615,0.002320,0.011784,0.064666,0.993446,-0.092444,-0.013877
5,0.000163,0.000614,0.003109,0.016673,0.088190,0.983436,-0.157444
6,0.000052,0.000198,0.001000,0.005363,0.027900,0.155487,0.987429


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(cov_df.shape[0]), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0011601097274113181,
 0.001986466224989991,
 0.0020931258004039943,
 0.0034551344600391998,
 0.006553881703814657,
 0.016564151536231475,
 0.012571182194001773]